In [1]:
# imports
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text

In [2]:
# Database credentials
username = 'postgres'
password = 'hellosql'
host = 'localhost'
port = '5433'
database = 'da_project_streaming'

# Connection String
connection_string = f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}'

# Engine
engine = create_engine(connection_string)

In [3]:
# select all from amazon database
query = "SELECT * FROM amazon_prime_titles"
df_amazon = pd.DataFrame(engine.connect().execute(text(query)))

In [4]:
df_amazon.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,None,113 min,"Comedy, Drama",A small fishing village must procure a local d...
1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...
2,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,"March 30, 2021",2017,None,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...
3,s4,Movie,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",United States,"March 30, 2021",2014,None,69 min,Documentary,"Pink breaks the mold once again, bringing her ..."
4,s5,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",United Kingdom,"March 30, 2021",1989,None,45 min,"Drama, Fantasy",Teenage Matt Banting wants to work with a famo...


In [5]:
df_amazon.duplicated().sum() # No duplicated values

0

In [6]:
df_amazon.isna().sum()

show_id            0
type               0
title              0
director        2082
cast            1233
country         8996
date_added      9513
release_year       0
rating           337
duration           0
listed_in          0
description        0
dtype: int64

In [7]:
df_amazon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9668 entries, 0 to 9667
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       9668 non-null   object
 1   type          9668 non-null   object
 2   title         9668 non-null   object
 3   director      7586 non-null   object
 4   cast          8435 non-null   object
 5   country       672 non-null    object
 6   date_added    155 non-null    object
 7   release_year  9668 non-null   int64 
 8   rating        9331 non-null   object
 9   duration      9668 non-null   object
 10  listed_in     9668 non-null   object
 11  description   9668 non-null   object
dtypes: int64(1), object(11)
memory usage: 906.5+ KB


In [8]:
# drop some null values
df_amazon.dropna(subset='rating', inplace=True)
df_amazon.drop('date_added', axis=1, inplace=True) # date added mostly null so drop
df_amazon = df_amazon.fillna('') # fill na with empty string values
df_amazon.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9331 entries, 1 to 9667
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       9331 non-null   object
 1   type          9331 non-null   object
 2   title         9331 non-null   object
 3   director      9331 non-null   object
 4   cast          9331 non-null   object
 5   country       9331 non-null   object
 6   release_year  9331 non-null   int64 
 7   rating        9331 non-null   object
 8   duration      9331 non-null   object
 9   listed_in     9331 non-null   object
 10  description   9331 non-null   object
dtypes: int64(1), object(10)
memory usage: 874.8+ KB


In [9]:
df_amazon['duration'].unique()

array(['110 min', '1 Season', '87 min', '104 min', '62 min', '94 min',
       '50 min', '3 Seasons', '2 Seasons', '6 min', '86 min', '36 min',
       '37 min', '103 min', '9 min', '18 min', '14 min', '20 min',
       '19 min', '22 min', '60 min', '54 min', '5 min', '84 min',
       '126 min', '125 min', '109 min', '89 min', '85 min', '40 min',
       '111 min', '33 min', '34 min', '95 min', '99 min', '78 min',
       '4 Seasons', '77 min', '55 min', '53 min', '115 min', '58 min',
       '49 min', '135 min', '91 min', '64 min', '131 min', '59 min',
       '52 min', '122 min', '92 min', '90 min', '102 min', '65 min',
       '114 min', '56 min', '136 min', '70 min', '138 min', '113 min',
       '100 min', '480 min', '4 min', '30 min', '152 min', '68 min',
       '57 min', '96 min', '7 Seasons', '98 min', '88 min', '31 min',
       '151 min', '149 min', '9 Seasons', '35 min', '141 min', '121 min',
       '48 min', '140 min', '46 min', '51 min', '106 min', '93 min',
       '27 min', '69 min

In [10]:
df_amazon['rating'].unique()

array(['13+', 'ALL', '18+', 'R', 'TV-Y', 'TV-Y7', 'NR', '16+', 'TV-PG',
       '7+', 'TV-14', 'TV-NR', 'TV-G', 'PG-13', 'TV-MA', 'G', 'PG',
       'NC-17', 'UNRATED', '16', 'AGES_16_', 'AGES_18_', 'ALL_AGES',
       'NOT_RATE'], dtype=object)

In [11]:
# Fix rating for shows so there are not multiple that mean the same thing
def fix_ratings(string):
    if string == 'ALL_AGES' or string == 'ALL':
        return 'G'
    elif string == 'UNRATED' or string == 'NOT_RATE':
        return 'NR'
    elif string == '16' or string == 'AGES_16_':
        return '16+'
    elif string == 'AGES_18_':
        return '18+'
    else:
        return string

In [12]:
# apply function to normalize ratings
df_amazon['rating'] = df_amazon['rating'].apply(fix_ratings)
df_amazon['rating'].unique()

array(['13+', 'G', '18+', 'R', 'TV-Y', 'TV-Y7', 'NR', '16+', 'TV-PG',
       '7+', 'TV-14', 'TV-NR', 'TV-G', 'PG-13', 'TV-MA', 'PG', 'NC-17'],
      dtype=object)

In [15]:
df_amazon['country'].unique()

array(['India', '', 'Italy', 'United States', 'United Kingdom, France',
       'United States, Italy', 'United States, India',
       'United Kingdom, United States',
       'United States, United Kingdom, Germany', 'Japan',
       'China, United States, United Kingdom',
       'Denmark, United Kingdom, Czech Republic, Netherlands',
       'United Kingdom', 'Canada', 'United States, Ireland',
       'United States, United Kingdom, Canada',
       'United Kingdom, United States, India', 'United Kingdom, India',
       'United States, China', 'Canada, United States',
       'Brazil, United States, India', 'United States, Thailand', 'Spain',
       'Switzerland, India', 'United Kingdom, Canada, United States',
       'Germany', 'Australia', 'Belgium',
       'Chile, Argentina, United States', 'Mexico',
       'Denmark, Sweden, United States', 'Netherlands',
       'United States, United Kingdom', 'New Zealand',
       'United States, Portugal', 'United States, France',
       'Canada, Uni

In [22]:
def fix_countries(string):
    return string.replace('\'',  '').strip()

df_amazon['country'] = df_amazon['country'].apply(fix_countries)
df_amazon['country'].unique()

array(['India', '', 'Italy', 'United States', 'United Kingdom, France',
       'United States, Italy', 'United States, India',
       'United Kingdom, United States',
       'United States, United Kingdom, Germany', 'Japan',
       'China, United States, United Kingdom',
       'Denmark, United Kingdom, Czech Republic, Netherlands',
       'United Kingdom', 'Canada', 'United States, Ireland',
       'United States, United Kingdom, Canada',
       'United Kingdom, United States, India', 'United Kingdom, India',
       'United States, China', 'Canada, United States',
       'Brazil, United States, India', 'United States, Thailand', 'Spain',
       'Switzerland, India', 'United Kingdom, Canada, United States',
       'Germany', 'Australia', 'Belgium',
       'Chile, Argentina, United States', 'Mexico',
       'Denmark, Sweden, United States', 'Netherlands',
       'United States, United Kingdom', 'New Zealand',
       'United States, Portugal', 'United States, France',
       'Canada, Uni